<h1><center>Προχωρημένα Θέματα Βάσεων Δεδομένων</center></h1>
<h2><center>Εξαμηνιαία Εργασία</center></h2>
<hr>

| Ονοματεπώνυμο      | Αριθμός Μητρώου |
| ------------------ | --------------- |
| Δοντάς Σπυρίδων    | 03114141        |
| Τσιούρβας Αστέριος | 03114133        |

<h1><center>Ερώτημα 1</center></h1>
<hr>

In [1]:
import datetime as dt

#### Read Data to RDDs

In [2]:
vendors = sc.textFile("/Project/yellow_tripvendors_1m.csv")
data = sc.textFile("/Project/yellow_tripdata_1m.csv")

#### Key - Value break for trip duration

In [3]:
def start_duration(line):
    """Function to map over RDD lines.
    
    Produces the necessary tuples for key - value processing.
    The starttime of the trip becomes the key, while the duration
    of the trip becomes the value.
    
    Parameters
    ----------
    
    line: string
        CSV delimited string from RDD.
        
    Returns
    -------
    
    keyval: tuple of (string, float)
        Key   -> starttime
        Value -> duration
    """
    contents = line.split(",")
    start, end = contents[1], contents[2]
    starttime = dt.datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
    endtime = dt.datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    diff = endtime - starttime
    diff = diff.total_seconds() / 60
    return "{:02d}".format(starttime.hour), diff

#### Map data

In [4]:
ndata = data.map(start_duration)

#### Find Average Duration per StartTime
* make each value to a tuple of (value, 1)
* reduce by key summing all tuples per element
  * now for each key we have a tuple with `(sum(values), num(values))`
* map each value tuple `(sum, len)` to `sum / num` (which is average)

In [5]:
ndata = ndata.mapValues(lambda x: (x, 1))\
             .reduceByKey(lambda a, b: tuple(map(sum, zip(a, b))))\
             .mapValues(lambda x: x[0] / x[1])

#### Save File
* convert to dataframe with headers
* gather to one partition
* save as csv

In [6]:
df = ndata.toDF(["HourOfDay", "AverageTripDuration"])\
          .sort("HourOfDay")
df.coalesce(1)\
  .write\
  .csv("/Project/TripDuration", header="true", mode="overwrite")

#### Show Results

In [7]:
df.show(30)

+---------+-------------------+
|HourOfDay|AverageTripDuration|
+---------+-------------------+
|       00|  14.01779373736224|
|       01| 13.975069898907156|
|       02| 13.035635592676691|
|       03| 13.322282520526896|
|       04| 13.799857931121979|
|       05| 13.275583221175415|
|       06| 12.487420237563251|
|       07| 13.395006418527384|
|       08| 14.627504543367854|
|       09| 14.670106419765608|
|       10|  14.65793916969826|
|       11| 14.935821221905574|
|       12| 15.130881322885706|
|       13| 15.553918733195145|
|       14| 16.523138380789458|
|       15| 30.223498632126187|
|       16|  17.21307206937467|
|       17| 16.510825654408585|
|       18|  15.29045374860121|
|       19| 14.221208805985764|
|       20| 13.575899636364237|
|       21| 13.510855327392878|
|       22| 14.231797625637382|
|       23| 13.958471125232714|
+---------+-------------------+



#### Key - Value break for price

In [8]:
def id_price(line):
    """Function to map over RDD lines.
    
    Produces the necessary tuples for key - value processing.
    The id of the trip becomes the key, while the price of
    the trip becomes the value.
    
    Parameters
    ----------
    
    line: string
        CSV delimited string from RDD.
        
    Returns
    -------
    
    idprice: tuple of (string, float)
        Key   -> id of trip
        Value -> price of trip
    """
    contents = line.split(",")
    return contents[0], float(contents[7])

#### Map `vendors` and `data`

In [9]:
# just make a tuple of the two values (trip id, vendor id)
tvendors = vendors.map(lambda x: tuple(x.split(",")))
prices = data.map(id_price)

#### Inner Join `vendors` and `prices` based on trip id

In [10]:
tvendors = tvendors.toDF(["id", "company"])
prices = prices.toDF(["id", "price"])
prices = tvendors.join(prices, on="id", how="inner")
prices = prices.rdd

#### Find max price per vendor id
* map rdd to tuples of `(vendor id, price)`
  * rdd after join has `Row` of `(id, company, price)`
* reduce by key finding `max`

In [11]:
prices = prices.map(lambda x: (x["company"], x["price"]))\
               .reduceByKey(max)

#### Save File
* convert to dataframe with headers
* gather to one partition
* save as csv

In [12]:
df = prices.toDF(["VendorID", "MaxAmountPaid"])
df.coalesce(1)\
  .write\
  .csv("/Project/MaxAmountPaid", header="true", mode="overwrite")

#### Show Results

In [13]:
df.show()

+--------+-------------+
|VendorID|MaxAmountPaid|
+--------+-------------+
|       1|    503326.33|
|       2|    548463.35|
+--------+-------------+

